In [1]:
from dotenv import load_dotenv
import os

import pandas as pd
from openai import OpenAI

from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

In [2]:
load_dotenv()

True

In [3]:
test = pd.read_csv('../csv/tweet-tone-test.csv')[0:300]
test

,Tweets,Label
0,"Hayatın bana yaptığı en güzel sürpriz, sen oldun",Sürpriz
1,Yakında patlayacağım ama volkanik bir patlama ...,Korku
2,Artık korkuyorum çok özel ve değerli olduğunu ...,Korku
3,"Haziran ayına gelince ‘'sınav Mart'daydı, kaçı...",Korku
4,Kar görünce sevinen panda kadar sevinçliyim şu an,Mutlu
...,...,...
295,Şu an gerçekten tam bir sinir küpüyüm,Kızgın
296,Arabanın anahtarı annemde gitmiş Allah’ım kafa...,Kızgın
297,Şu 'Kore dizisi ne ya' diyen insanlara sinir o...,Kızgın
298,44 mürettebatı bulunan Arjantin denizaltısında...,Korku


In [4]:
client = OpenAI(
    api_key=os.getenv('API_KEY'),
)


def classify_text(dataset):

    i = 0
    predictions = []
    while i < dataset.shape[0]:

        try:

            message = dataset.iloc[i]["Tweets"]
            prompt = f"""Do tone analysis on user text with labels: Kızgın, Korku, Mutlu, Sürpriz or Üzgün. Message: '{message}'.
The output should only contain one of the five words: Kızgın, Korku, Mutlu, Sürpriz or Üzgün.
"""
            chat_completion = client.chat.completions.create(
                messages=[
                    {"role": "user", "content": prompt}
                ],
                model="ft:gpt-3.5-turbo-1106:personal::8mOcSIg2",
            )

            response =  chat_completion.choices[0].message.content
            predictions.append(response)

        except Exception as err:
            print(err)

        
        i = i + 1
        print(f"Total messages processed: {i}")
    return predictions

In [5]:
predictions = classify_text(test)

Total messages processed: 1
Total messages processed: 2
Total messages processed: 3
Total messages processed: 4
Total messages processed: 5
Total messages processed: 6
Total messages processed: 7
Total messages processed: 8
Total messages processed: 9
Total messages processed: 10
Total messages processed: 11
Total messages processed: 12
Total messages processed: 13
Total messages processed: 14
Total messages processed: 15
Total messages processed: 16
Total messages processed: 17
Total messages processed: 18
Total messages processed: 19
Total messages processed: 20
Total messages processed: 21
Total messages processed: 22
Total messages processed: 23
Total messages processed: 24
Total messages processed: 25
Total messages processed: 26
Total messages processed: 27
Total messages processed: 28
Total messages processed: 29
Total messages processed: 30
Total messages processed: 31
Total messages processed: 32
Total messages processed: 33
Total messages processed: 34
Total messages processe

In [6]:
predictions

['Sürpriz',
 'Korku',
 'Korku',
 'Korku',
 'Mutlu',
 'Sürpriz',
 'Kızgın',
 'Kızgın',
 'Üzgün',
 'Kızgın',
 'Kızgın',
 'Kızgın',
 'Sürpriz',
 'Üzgün',
 'Sürpriz',
 'Kızgın',
 'Üzgün',
 'Sürpriz',
 'Korku',
 'Sürpriz',
 'Üzgün',
 'Sürpriz',
 'Kızgın',
 'Korku',
 'Kızgın',
 'Korku',
 'Kızgın',
 'Korku',
 'Korku',
 'Üzgün',
 'Mutlu',
 'Mutlu',
 'Kızgın',
 'Kızgın',
 'Mutlu',
 'Üzgün',
 'Sürpriz',
 'Sürpriz',
 'Üzgün',
 'Üzgün',
 'Üzgün',
 'Sürpriz',
 'Üzgün',
 'Üzgün',
 'Üzgün',
 'Sürpriz',
 'Korku',
 'Sürpriz',
 'Mutlu',
 'Korku',
 'Korku',
 'Sürpriz',
 'Sürpriz',
 'Korku',
 'Kızgın',
 'Mutlu',
 'Sürpriz',
 'Mutlu',
 'Mutlu',
 'Mutlu',
 'Üzgün',
 'Korku',
 'Mutlu',
 'Üzgün',
 'Mutlu',
 'Mutlu',
 'Kızgın',
 'Üzgün',
 'Korku',
 'Kızgın',
 'Kızgın',
 'Sürpriz',
 'Mutlu',
 'Kızgın',
 'Sürpriz',
 'Üzgün',
 'Mutlu',
 'Mutlu',
 'Korku',
 'Üzgün',
 'Mutlu',
 'Sürpriz',
 'Üzgün',
 'Korku',
 'Mutlu',
 'Sürpriz',
 'Kızgın',
 'Üzgün',
 'Kızgın',
 'Korku',
 'Üzgün',
 'Korku',
 'Korku',
 'Üzgün',
 'Kı

In [7]:
pred_df = pd.DataFrame(predictions, columns=['Label'])
pred_df.head(10)

,Label
0,Sürpriz
1,Korku
2,Korku
3,Korku
4,Mutlu
5,Sürpriz
6,Kızgın
7,Kızgın
8,Üzgün
9,Kızgın


In [8]:
def change_label(label):
    if 'Kızgın' in label:
        return 0
    elif 'Korku' in label:
        return 1
    elif 'Mutlu' in label:
        return 2
    elif 'Sürpriz' in label:
        return 3
    elif 'Üzgün' in label:
        return 4

In [9]:
pred_df['Label'] = pred_df['Label'].map(change_label)
pred_df

,Label
0,3
1,1
2,1
3,1
4,2
...,...
295,0
296,0
297,0
298,1


In [10]:
test['Label'] = test['Label'].map({'Kızgın': 0, 'Korku': 1, 'Mutlu': 2, 'Sürpriz':3, 'Üzgün':4})

In [11]:
pred_df[pred_df['Label'].isna()]

,Label


In [12]:
predictions[143]

'Sürpriz'

In [51]:
pred_df.iloc[143] = 3

Label    3.0
Name: 143, dtype: float64

In [13]:
predictions[217]

'Üzgün'

In [56]:
pred_df = pred_df.drop(217)

In [57]:
test = test.drop(217)

In [14]:
print(classification_report(pred_df['Label'].values, test['Label'].values))
print(accuracy_score(pred_df['Label'].values, test['Label'].values))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        57
           1       0.97      1.00      0.98        57
           2       0.98      1.00      0.99        58
           3       0.98      1.00      0.99        61
           4       1.00      0.93      0.96        67

    accuracy                           0.98       300
   macro avg       0.98      0.99      0.98       300
weighted avg       0.98      0.98      0.98       300

0.9833333333333333
